In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from JM_animals_CRUD_altered import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable Only')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1("Here's the header identifier, from Jacob Mousseau"))),
    html.Center(html.P('Select up to five options from the table for the map')),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                #############################################################
                #Changed the values here to exemplify what each section is.
                #############################################################
               {'label': 'Water Rescue', 'value': 'WTR-R'},
               {'label': 'Mountain Rescue', 'value': 'MTN-R'},
               {'label': 'Disaster Rescue', 'value': 'DIS-R'},
               {'label': 'Reset','value': 'RESET'}
            ],
            value='RESET',
            labelStyle={'display': 'inline-block'}
       )  
    ),

    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        style_table={'overflowX': 'auto'},
        style_cell={
        'height': 'auto',
        # all three widths are needed
        'whiteSpace': 'normal'
    },
        data=df.to_dict('records'),
#Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            #dcc.Graph(id='graph-id'),
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id',"data"),
    Output('datatable-id','columns')],
    [Input('filter-type', 'value')]
)
#######################################################################################################################################
#This was originially on_click, but I changed it to make more sense with the rest of the code, and re-wrote it accordingly.
#I also cleaned up the organization of each statement to make it easier to read.
#######################################################################################################################################
def update_dashboard(filter_type):
    
    
#Value 1/WTR-R
        if filter_type == 'WTR-R':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                    {'$or': [
                                        {'breed': 'Labrador Retriever Mix'},
                                        {'breed': 'Chesa Bay Retr Mix'},
                                        {'breed': 'Newfoundland Mix'},
                                        {'breed': 'Newfoundland/Labrador Retriever'},
                                        {'breed': 'Newfoundland/Australian Cattle Dog'},
                                        {'breed': 'Newfoundland/Great Pyrenees'}]
                                    },
                                    {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                            {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                    }]
                        })))
#Value 2/MTN-R
        elif filter_type == 'MTR-R':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                    {'$or': [
                                        {'breed': 'German Shepherd'},
                                        {'breed': 'Alaskan Malamute'},
                                        {'breed': 'Old English Sheepdog'},
                                        {'breed': 'Rottweiler'},
                                        {'breed': 'Siberian Husky'}]
                                    },
                                    {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                            {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                    }]
                            })))
#Value 3/DIS-R
elif filter_type == 'DIS-R':
            #breeds and ages
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                    {'$or': [
                                        {'breed': 'Doberman Pinscher'},
                                        {'breed': 'German Shepherd'},
                                        {'breed': 'Golden Retriever'},
                                        {'breed': 'Bloodhound'},
                                        {'breed': 'Rottweiler'}]
                                    },
                                    {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                            {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                    }]
                                                })))
#Value 4/RESET
        elif filter_type == 'RESET':
            df = pd.DataFrame.from_records(shelter.read({}))

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
#############################################################
#This was also restructured for better readability. 
#############################################################
def generate_chart(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]
@app.callback(
    #Added additional callbacks for re-doing the update map section.
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'selected_columns')])
def update_map(viewData):
    #########################################################################################################################################
    #The original version of the update_map code is commented out below, the instructor for this course helped me with it significantly
    #And I figured a good way to show my progress would not be by using chunks of code the instructor gave me, so I restructured this part,
    #########################################################################################################################################
##FIXME: Add in the code for your geolocation chart
##If you completed the Module Six Assignment, you can copy in the code you created here.
#    #viewDF = pd.DataFrame.from_dict(viewData)
#    #dff = viewDF.loc[rows]
#    #os.write(1,str(dff.loc[0,'location_long']).strip('[]').encode())
#    dff = pd.DataFrame.from_dict(viewData)
#    # Austin TX is at [30.75,-97.48]
#    return [
#        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
#            dl.TileLayer(id="base-layer-id"),
#            # Marker with tool tip and popup
#            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
#                dl.Tooltip(dff['breed']),
#                dl.Popup([
#                    html.H1("Animal Name"),
#                    html.P(dff.loc[0,'name'])
#                ])
#            ])
#        ])
#    ]

dff = pd.DataFrame.from_dict(viewData)
if selected_rows == []:
        selected_rows = [0]
        
        #This code, using the old section as a jumping off point, should create a map for a single(or default) row. 
        if len(selected_rows) == 1:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ])
            ])
        ]
            #########################################################################################################################################
            #I could add more map marker setups for more sections of what the table would be.
            #########################################################################################################################################
        
        
        
        
app